# Exploratory Data Analysis

Understanding the dataset we have to explore how the data is present in the database and if there is any need to create new features that can help us with:
1. Who is churning, and more importantly, why?
2. Digital adoption as a churn-prevention lever

In [1]:
import pandas as pd
import numpy as np

In [6]:
from src.postgres_session import get_postgres_connection
conn = get_postgres_connection()

ModuleNotFoundError: No module named 'src'

In [69]:
df = pd.read_sql("select * from bank_customer_churn", conn)

In [70]:
df

,Acct_ID,YEARS_WITH_BANK,CHURN,RISK_RATING,CURRENCY,AVE BAL,SCHEME_TYPE,MOBILE_APP_ADOPTION,INTERNET_BANKING_ADOPTION,USSD_BANKING_ADOPTION,DIGITAL_LOAN,UNSECURED_LOAN,TERMLOAN_STATUS,CREDIT_CARD,SUBSEGMENT,LAST_12_MONTHS_CREDIT_VOLUME,LAST_12_MONTHS_DEBIT_VOLUME,LAST_12_MONTHS_DEBIT_VALUE,LAST_12_MONTHS_CREDIT_VALUE
0,Account_0,10,0,LOW,NGN,302.34,ODA,N,N,N,N,N,N,N,product7,0,0,-,-
1,Account_1,7,1,LOW,NGN,640.55,SBA,Y,Y,N,N,N,N,N,product7,23,84,"1,641,781.54","1,650,610.00"
2,Account_2,12,1,MEDIUM,NGN,"2,549.96",SBA,N,N,Y,N,N,N,N,product6,0,2,"5,427.50",-
3,Account_3,12,0,MEDIUM,USD,"9,999.17",ODA,N,Y,Y,N,N,N,N,product6,0,0,-,-
4,Account_4,12,1,MEDIUM,NGN,455.25,SBA,Y,Y,Y,N,N,N,N,product8,439,116,"6,768,632.00","6,784,830.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,Account_489070,8,0,MEDIUM,NGN,106.60,ODA,N,N,N,N,N,N,N,product7,0,0,-,-
499996,Account_489071,9,1,LOW,NGN,33.72,SBA,N,N,Y,N,N,N,N,product5,142,91,"296,204.03","294,710.00"
499997,Account_489072,4,0,LOW,NGN,60.58,SBA,Y,N,Y,N,N,N,N,product5,0,0,-,-
499998,Account_489073,3,0,LOW,NGN,"1,151.76",SBA,Y,Y,N,N,N,N,N,product5,0,0,-,-


In [71]:
df.columns = df.columns.str.lower()

In [72]:
df = df.rename(columns={
  "years_with_bank" : "years",
  "risk_rating" : "risk",
  "scheme_type" : "scheme",
  "mobile_app_adoption" : "mobile_app",
  "internet_banking_adoption" : "internet_banking",
  "ussd_banking_adoption" : "ussd_banking",
  "termloan_status" : "termloan",
  "last_12_months_credit_volume" : "credit_vol",
  "last_12_months_debit_volume" : "debit_vol",
  " last_12_months_debit_value " : "debit_val",
  " last_12_months_credit_value " : "credit_val",
  " ave bal " : "ave_bal"
}
)

In [73]:
df.columns

Index(['acct_id', 'years', 'churn', 'risk', 'currency', 'ave_bal', 'scheme',
       'mobile_app', 'internet_banking', 'ussd_banking', 'digital_loan',
       'unsecured_loan', 'termloan', 'credit_card', 'subsegment', 'credit_vol',
       'debit_vol', 'debit_val', 'credit_val'],
      dtype='object')

In [74]:
df.dtypes

acct_id             object
years                int64
churn                int64
risk                object
currency            object
ave_bal             object
scheme              object
mobile_app          object
internet_banking    object
ussd_banking        object
digital_loan        object
unsecured_loan      object
termloan            object
credit_card         object
subsegment          object
credit_vol           int64
debit_vol            int64
debit_val           object
credit_val          object
dtype: object

In [75]:
df.isnull().sum()

acct_id             0
years               0
churn               0
risk                1
currency            0
ave_bal             0
scheme              0
mobile_app          0
internet_banking    0
ussd_banking        0
digital_loan        0
unsecured_loan      0
termloan            0
credit_card         0
subsegment          0
credit_vol          0
debit_vol           0
debit_val           0
credit_val          0
dtype: int64

In [76]:
df['ave_bal'] = (
    df['ave_bal']
        .astype(str)
        .str.replace(',', '', regex=False)
        .str.replace(r'\((.*?)\)', r'-\1', regex=True)
        .str.strip()
        .replace(['', '-'], np.nan)
        .astype(float)
)


In [ ]:
#Cleaning debit value column and converting to float
df['debit_val'] = (
    df['debit_val']
        .astype(str)
        .str.replace(',', '', regex=False)
        .str.replace(r'\((.*?)\)', r'-\1', regex=True)
        .str.strip()
        .replace(['', '-'], 0)
        .astype(float)
)

In [ ]:
#Cleaning credit value column and converting to float
df['credit_val'] = (
    df['credit_val']
        .astype(str)
        .str.replace(',', '', regex=False)
        .str.replace(r'\((.*?)\)', r'-\1', regex=True)
        .str.strip()
        .replace(['', '-'], 0)
        .astype(float)
)

In [46]:
df.dtypes

acct_id              object
years                 int64
churn                 int64
risk                 object
currency             object
ave_bal             float64
scheme               object
mobile_app           object
internet_banking     object
ussd_banking         object
digital_loan         object
unsecured_loan       object
termloan             object
credit_card          object
subsegment           object
credit_vol            int64
debit_vol             int64
debit_val           float64
credit_val          float64
dtype: object

In [47]:
df.isnull().sum()

acct_id                 0
years                   0
churn                   0
risk                    1
currency                0
ave_bal             66013
scheme                  0
mobile_app              0
internet_banking        0
ussd_banking            0
digital_loan            0
unsecured_loan          0
termloan                0
credit_card             0
subsegment              0
credit_vol              0
debit_vol               0
debit_val               0
credit_val              0
dtype: int64

In [48]:
df = df.dropna()

In [49]:
df.shape

(433986, 19)

In [50]:
df

,acct_id,years,churn,risk,currency,ave_bal,scheme,mobile_app,internet_banking,ussd_banking,digital_loan,unsecured_loan,termloan,credit_card,subsegment,credit_vol,debit_vol,debit_val,credit_val
0,Account_0,10,0,LOW,NGN,302.34,ODA,N,N,N,N,N,N,N,product7,0,0,0.00,0.0
1,Account_1,7,1,LOW,NGN,640.55,SBA,Y,Y,N,N,N,N,N,product7,23,84,1641781.54,1650610.0
2,Account_2,12,1,MEDIUM,NGN,2549.96,SBA,N,N,Y,N,N,N,N,product6,0,2,5427.50,0.0
3,Account_3,12,0,MEDIUM,USD,9999.17,ODA,N,Y,Y,N,N,N,N,product6,0,0,0.00,0.0
4,Account_4,12,1,MEDIUM,NGN,455.25,SBA,Y,Y,Y,N,N,N,N,product8,439,116,6768632.00,6784830.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499994,Account_489069,5,0,MEDIUM,NGN,3046.39,SBA,N,N,N,N,N,N,N,product6,0,0,0.00,0.0
499995,Account_489070,8,0,MEDIUM,NGN,106.60,ODA,N,N,N,N,N,N,N,product7,0,0,0.00,0.0
499996,Account_489071,9,1,LOW,NGN,33.72,SBA,N,N,Y,N,N,N,N,product5,142,91,296204.03,294710.0
499997,Account_489072,4,0,LOW,NGN,60.58,SBA,Y,N,Y,N,N,N,N,product5,0,0,0.00,0.0


In [51]:
cols = ['acct_id', 'risk', 'currency', 'scheme', 'mobile_app', 'internet_banking', 'ussd_banking', 'digital_loan', 'unsecured_loan', 'termloan', 'credit_card', 'subsegment']

df[cols] = df[cols].apply(lambda col: col.str.strip())

In [52]:
df['currency'].unique()

array(['NGN', 'USD', 'EUR', 'GBP', 'ZAR', 'CAD'], dtype=object)

In [53]:
# One hot encoding
cols = [
    "mobile_app", "internet_banking", "ussd_banking",
    "digital_loan", "unsecured_loan", "termloan", "credit_card"
]

df[cols] = df[cols].replace({"Y": 1, "N": 0}).astype(int)

C:\Users\abhis\AppData\Local\Temp\ipykernel_22000\2569711480.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[cols] = df[cols].replace({"Y": 1, "N": 0}).astype(int)


In [54]:
df

,acct_id,years,churn,risk,currency,ave_bal,scheme,mobile_app,internet_banking,ussd_banking,digital_loan,unsecured_loan,termloan,credit_card,subsegment,credit_vol,debit_vol,debit_val,credit_val
0,Account_0,10,0,LOW,NGN,302.34,ODA,0,0,0,0,0,0,0,product7,0,0,0.00,0.0
1,Account_1,7,1,LOW,NGN,640.55,SBA,1,1,0,0,0,0,0,product7,23,84,1641781.54,1650610.0
2,Account_2,12,1,MEDIUM,NGN,2549.96,SBA,0,0,1,0,0,0,0,product6,0,2,5427.50,0.0
3,Account_3,12,0,MEDIUM,USD,9999.17,ODA,0,1,1,0,0,0,0,product6,0,0,0.00,0.0
4,Account_4,12,1,MEDIUM,NGN,455.25,SBA,1,1,1,0,0,0,0,product8,439,116,6768632.00,6784830.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499994,Account_489069,5,0,MEDIUM,NGN,3046.39,SBA,0,0,0,0,0,0,0,product6,0,0,0.00,0.0
499995,Account_489070,8,0,MEDIUM,NGN,106.60,ODA,0,0,0,0,0,0,0,product7,0,0,0.00,0.0
499996,Account_489071,9,1,LOW,NGN,33.72,SBA,0,0,1,0,0,0,0,product5,142,91,296204.03,294710.0
499997,Account_489072,4,0,LOW,NGN,60.58,SBA,1,0,1,0,0,0,0,product5,0,0,0.00,0.0


In [55]:
df['risk'].unique()

array(['LOW', 'MEDIUM', 'HIGH', 'C-LOW RISK', 'A-HIGH RISK', 'B-LOW RISK'],
      dtype=object)

In [56]:
df['risk'] = df['risk'].replace({"A-HIGH RISK": 'HIGH', "B-LOW RISK": 'MEDIUM', 'C-LOW RISK': 'LOW'})

- The table contains 

---

- Since we need to find out the impact of digital banking on customer churn we need to create a column with no. of digital channels used per account_id.

In [57]:
cols = ["mobile_app", "internet_banking", "ussd_banking"]
df['digital_channels_used'] = df[cols].sum(axis=1)

In [58]:
churn_rate = round(df['churn'].sum()/df['churn'].count()*100, 2)
print(f"The Churn Rate is {churn_rate}%")

The Churn Rate is 55.08%


In [60]:
df.columns

Index(['acct_id', 'years', 'churn', 'risk', 'currency', 'ave_bal', 'scheme',
       'mobile_app', 'internet_banking', 'ussd_banking', 'digital_loan',
       'unsecured_loan', 'termloan', 'credit_card', 'subsegment', 'credit_vol',
       'debit_vol', 'debit_val', 'credit_val', 'digital_channels_used'],
      dtype='object')

In [62]:
from sqlalchemy import text

engine = get_postgres_connection()

create_table_sql = """
CREATE TABLE IF NOT EXISTS customer_churn_summary (
    acct_id VARCHAR(100),
    years INTEGER,
    churn INTEGER,
    risk VARCHAR(10),
    currency VARCHAR(10),
    ave_bal FLOAT,
    scheme VARCHAR(10),
    mobile_app INTEGER,
    internet_banking INTEGER,
    ussd_banking INTEGER,
    digital_loan INTEGER,
    unsecured_loan INTEGER,
    termloan INTEGER,
    credit_card INTEGER,
    subsegment VARCHAR(15),
    credit_vol FLOAT,
    debit_vol FLOAT,
    debit_val FLOAT,
    credit_val FLOAT,
    digital_channels_used INTEGER,
    PRIMARY KEY (acct_id)
);
"""

with engine.begin() as conn:
    conn.execute(text(create_table_sql))

In [64]:
conn = get_postgres_connection()
pd.read_sql("""select * from customer_churn_summary""", conn)

,acct_id,years,churn,risk,currency,ave_bal,scheme,mobile_app,internet_banking,ussd_banking,digital_loan,unsecured_loan,termloan,credit_card,subsegment,credit_vol,debit_vol,debit_val,credit_val,digital_channels_used


In [65]:
df.to_sql('customer_churn_summary', con=conn, if_exists= 'replace', index=False)

986

In [66]:
pd.read_sql("""select * from customer_churn_summary""", conn)

,acct_id,years,churn,risk,currency,ave_bal,scheme,mobile_app,internet_banking,ussd_banking,digital_loan,unsecured_loan,termloan,credit_card,subsegment,credit_vol,debit_vol,debit_val,credit_val,digital_channels_used
0,Account_0,10,0,LOW,NGN,302.34,ODA,0,0,0,0,0,0,0,product7,0,0,0.00,0.0,0
1,Account_1,7,1,LOW,NGN,640.55,SBA,1,1,0,0,0,0,0,product7,23,84,1641781.54,1650610.0,2
2,Account_2,12,1,MEDIUM,NGN,2549.96,SBA,0,0,1,0,0,0,0,product6,0,2,5427.50,0.0,1
3,Account_3,12,0,MEDIUM,USD,9999.17,ODA,0,1,1,0,0,0,0,product6,0,0,0.00,0.0,2
4,Account_4,12,1,MEDIUM,NGN,455.25,SBA,1,1,1,0,0,0,0,product8,439,116,6768632.00,6784830.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433981,Account_489069,5,0,MEDIUM,NGN,3046.39,SBA,0,0,0,0,0,0,0,product6,0,0,0.00,0.0,0
433982,Account_489070,8,0,MEDIUM,NGN,106.60,ODA,0,0,0,0,0,0,0,product7,0,0,0.00,0.0,0
433983,Account_489071,9,1,LOW,NGN,33.72,SBA,0,0,1,0,0,0,0,product5,142,91,296204.03,294710.0,1
433984,Account_489072,4,0,LOW,NGN,60.58,SBA,1,0,1,0,0,0,0,product5,0,0,0.00,0.0,2
